In [1]:
!pip install crewai
!pip install groq
!pip install langchain_groq
!pip install 'crewai[tools]'

ERROR: Could not find a version that satisfies the requirement crewai (from versions: none)
ERROR: No matching distribution found for crewai
ERROR: Could not find a version that satisfies the requirement groq (from versions: none)
ERROR: No matching distribution found for groq
ERROR: Could not find a version that satisfies the requirement langchain_groq (from versions: none)
ERROR: No matching distribution found for langchain_groq
ERROR: Invalid requirement: "'crewai[tools]'"


In [ ]:
from google.colab import userdata
groq_api_key=userdata.get('GROQ_API_KEY')
serper_api_key=userdata.get('SERPER_API_KEY')

In [ ]:
from langchain_groq import ChatGroq

# my LLM model - using groq for fast response - using llama3-8b model
llm = ChatGroq(groq_api_key=groq_api_key,
               model_name="llama3-8b-8192")

In [3]:
from crewai import Agent, Crew, Task, Process

ModuleNotFoundError: No module named 'crewai'

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeVideoSearchTool

# Initialize the tools
search_tool = SerperDevTool(serper_api_key=serper_api_key)
scrape_tool = ScrapeWebsiteTool()
# youtube_tool = YoutubeVideoSearchTool()

# Defining Agents

In [2]:
# Agent 1: attraction finder
attraction_finder = Agent(
    role="Activity & attraction finder",
    goal="Identify best places for travelling and exploring the given destination.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        """
        Your task is to find top places to travel at destination. These places might include adventure, cultural sites,spiritual places, food places, natural beauty, shopping. 
        Also find places based on traveller's Interest and activities.
        At the end make list of these places, for each place write 1) name of the place 2) Location 3) Average cost 4) type of the place(whether it is cultural site,adventure, spiritual place etc.) 5) average rating.
        """
    ),
    llm=ChatGroq(groq_api_key=groq_api_key,
               model_name="llama3-8b-8192")
)

NameError: name 'Agent' is not defined

In [5]:
# Agent 2: Itinerary planner
itinerary_planner = Agent(
    role="Itinerary planner",
    goal="Create day to day plan for traveller like on which day traveller will go to which places. Also find accommodation place to stay at each night.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    allow_delegation=True,
    backstory=(
        """
        Your task is to create day to day plan based on places found by Activity & attraction finder. For an example...
        Day-1 : traveller will visit this place,then this place, eat at this place, go to this park and stay at this place at night.
        Day-2 : traveller will have breakfast at this place,then go to this place,go to this temple and will have accommodation at this place.
        and so on.. for number of days
        Note that this is just an example or format. Means this way you have to create a day to day activity plan for traveller.
        Also try to keep place which are nearer to each other on same day, but don't keep too many places for one day and find accommodation place for that day based on accommodation preference of traveller.
        """
    ),
    llm=llm
)

NameError: name 'Agent' is not defined

In [ ]:
# Agent 3: Transportation Manager
transportation_manager = Agent(
    role="Transportation Manager",
    goal="Identify the means of transportations based on plan decided by Itinerary planner.",
    tools=[search_tool],
    verbose=True,
    allow_delegation=True,
    backstory=(
        """
        Find out the means of transportations and best paths for going to places decided by Itinerary planner.
        Try to minimize transportation time so that traveller can spend maximum time on enjoying various tourist places.
        Note that trip starts from starting point so also decide that how traveller will go from starting point to the destination.
        Also keep in mind his transportation preferences and find how he can go from one place to another place to maximise his comfort and enjoyment.
        Also communicate with Itinerary planner for better planning and then give final plan by adding that how traveller will travel to all these places in the plan made by Itinerary planner.
        """
    ),
    llm=llm
)

# Tasks

In [ ]:
activity_task = Task(
    description="""Find out best activities & attractions for travelling at destination {Destination} ,
                for number of days={Number of days}, number of persons={Number of persons} and
                by keeping in mind that traveller's interest of area is {Interests and activities}.""",
    expected_output="Best places to explore and travel at given destination and for each place I want 1) name of the place 2) Location 3) Average cost 4) type of the place(whether it is cultural site,adventure, spiritual place etc.) 5) average rating.
        ",
    agent=attraction_finder
)

In [ ]:
planning_task = Task(
    description="""Your job is to create best day by day plan based on places found by Activity & attraction finder.
                    This plan should contain every details about which places traveller will visit, where he will eat for lunch and dinner also where he will have accommodation.
                    Note that number of persons={Number of persons} and number of days in trip={Number of days}.""",
    expected_output="Detailed plan containing every details about which places traveller will visit, where he will eat for lunch and dinner and where he will stay for each day.",
    # async_execution=True,
    context=[activity_task],
    agent=itinerary_planner
)

In [ ]:
transportation_task = Task(
    description="""Your job is to add how traveller will do transportation to various places in plan decided by Itinerary planner.
                    Based on traveller's requirements Mode of transportation is by {Mode of transportation} and local transportation requirements are {Local transportation}.
                    Also note that starting point is {Trip starting point} and destination is {Destination}.
                    Comfortable Travellling dates are {Travel dates}.
                    Also note that first travellers will start their journey from starting point which is {Trip starting point} and then go to the destination {Destination} via {Mode of transportation}. And they will use local transportation = {Local transportation} to travel on various places at destination.
                    Note that number of persons={Number of persons} and number of days in trip={Number of days}.
                    """,
    expected_output="Final plan by adding how traveller will do transportation to various places in plan generated by Itinerary planner.",
    # async_execution=True,
    context=[planning_task],
    agent=transportation_manager
)

In [ ]:
# Trip starting point : the city or location where the trip begin
# Destination : The city or location where the trip is planned
# Number of days
# Number of persons
# Budget
# Travel dates : specific start & end dates
# Mode of Transportation : preferred means of travel (flight/train/car)
# Accommodation preferences : type of lodging (hotel/hostel/Airbnb)
# Interests and activities : Preferred activities or area of interests (adventure/cultural sites/relaxation/shopping)
# Meal preference : Dietary restrictions or allergic food or any meal preference
# Special requirements
# Local transportation : preference for getting around the destination (renting a car/using public transport/walking)

trip_panner_input = {
    'Trip starting point' : 'Surat,Gujarat,India',
    'Destination' : 'Ahmedabad,Gujarat,India',
    'Number of days' : '3',
    'Number of persons' : '2',
    'Mode of transportation' : 'train',
    'Accomodation preferences' : 'hotel',
    'Interests and activities' : '1st priority is exploring the place, 2nd priority is Food and 3rd priority is shopping',
    'Local transportation' : '1st priority is using public transport, 2nd priority is booking a cab and 3rd priority is walking'
}

In [ ]:
trip_planner_crew = Crew(
    agents=[attraction_finder,
            itinerary_planner,
            transportation_manager],

    tasks=[activity_task,
          planning_task,
          transportation_task],

    manager_llm=ChatGroq(groq_api_key=groq_api_key,
               model_name="mixtral-8x7b-32768"),

    process=Process.hierarchical,
    verbose=True
)

In [ ]:
result = trip_planner_crew.kickoff(inputs=trip_panner_input)

In [ ]:
from IPython.display import Markdown
Markdown(result)